Gauss Bonnet theorem on the first Dziuk’s surface
-------------------------------------------------

Consider the Dziuks surface with implicit equation as: 

\begin{equation*}
       \phi(x,y,z) = (x-z^2)^2 +y^2 +z^2-1 = 0
\end{equation*}
the Euler Characteristic is $\chi(\mathcal{M})=2$, therefore we have:

\begin{equation*}
\int_{\mathcal{M}}KdS = 4\pi
\end{equation*}
<img src="../images/dziuk's_surface.png" alt="drawing" width="300"/>


We utilize the `distmesh` library to generate a triangulation with $N_{\Delta}=390$ triangles for the Dziuk's. surface.

Imports

In [12]:
# Standard library imports:
import matplotlib.pyplot as plt
import numpy as np
from numba import njit
from time import time
import matplotlib.pyplot as plt
# Surfpy imports:
import sys
sys.path.append("../surf")
from surface_integration import integration

In [13]:
mesh_path ="../meshes/dziuk_N=3900.mat"
#zero level funtion of the Dziuk's surface
@njit(fastmath=True)
def phi(x: np.ndarray):
    return (x[0]-x[2]**2)**2+x[1]**2+x[2]**2-1
#gradient of the zero level funtion of the Dziuk's surface
@njit(fastmath=True)
def dphi(x: np.ndarray):
    return np.array([2*(x[0]-x[2]**2), 2*x[1], 2*(-2*x[0]*x[2]+2*x[2]**3+x[2])])

#Gauss curvature of the Dziuk's surface computed with Mathematica
def fun_1(x,y,z):
    return (y**2+z**2-(x-z**2)*(x*(2*x-1)+2*y**2+z**2-4*x*z**2+2*z**4))/(y**2+z**2+(x-z**2)*(x+(4*x-5)*z**2-4*z**4))**2


Error Evaluation Function

In [10]:
def err_t(integrand,intp_degree,mesh_path):
#     integrand = lambda x, y, z: 0*x+1
    t0 = time()
    num_result = integration(integrand,phi, dphi, mesh_path, intp_degree)
    t1 = time()
    exact_area =4*np.pi
    print("Relative error: ", abs(num_result - exact_area) / exact_area)
    print ("The main function takes:",{(t1-t0)})
    error=abs(num_result - exact_area) / exact_area
    time_s=t1-t0
    return error,time_s 

#error computed with dune
dune_error_2_15 =np.array([ 1.24939e-02, 2.11375e-04, 5.73064e-04, 9.15981e-06, 2.59706e-05,
 3.71289e-06, 8.00355e-07, 1.88131e-07, 3.22471e-07 ,9.25338e-07, 2.69146e-06,
 6.20421e-05, 5.06034e-06])
# running time of dune
running_times = np.array([1.391000e+00 ,3.140000e+00, 7.074000e+00, 1.596900e+01,
 3.083800e+01, 5.897300e+01 ,1.045970e+02 ,1.812150e+02, 2.935110e+02,
 4.702450e+02 ,7.800880e+02 ,1.153811e+03 ,1.670555e+03]) 

# Degree of Polynomial for surfpy
Nrange = list(range(3,30))
# Degree of Polynomial used for dune
Nrange_1 = list(range(3,16))
error1=[] 
execution_times = []
for n in Nrange:
    if n%1==0:print(n)
    erro1, times = err_t(fun_1,int(n),mesh_path)
    error1.append(erro1)
    execution_times.append(times)

# filename = "error_dziuk_N=3900.txt"

# # Write the error values to a text file
# with open(filename, "w") as file:
#     for error in error1:
#         file.write(f"{error},\n")
        
# Create subplots
fig, ax1 = plt.subplots(figsize=(7, 5))
# First plot
ax1.semilogy(Nrange, error1, '-or', label='HOSQ_CC')
ax1.semilogy(Nrange_1, dune_error_2_15, '-ob', label='DCG')
ax1.set_xlabel("Polynomial degree", fontsize=14)
ax1.set_ylabel("Relative error", fontsize=14)
ax1.legend(frameon=False, prop={'size': 10}, loc='best')
ax1.set_xticks(np.arange(min(Nrange), max(Nrange), 5))
ax1.set_ylim([1.0e-16, 1.0e-0])
ax1.grid(True, linestyle='--', alpha=0.7)

# The second plot
left, bottom, width, height = [0.55, 0.32, 0.35, 0.35]
ax2 = fig.add_axes([left, bottom, width, height])
ax2.plot(Nrange_1, running_times, '-*b')
ax2.plot(Nrange, execution_times, '-*r')
ax2.set_xlabel('Polynomial degree', fontsize=12)
ax2.set_ylabel('Runtime (sec)', fontsize=12)
ax2.set_xlim([2, 30])
ax2.set_ylim([0, 200])
ax2.grid(True, linestyle='--', alpha=0.7)
# ax1.yscale("log")
plt.savefig("../images/clenshaw_convergence_for_dziuk_linf.pdf", dpi=300, bbox_inches='tight')
# Show the plot
plt.show()